In [1]:
# Import our dependencies
import pandas as pd
import re

In [2]:
#  Import and read the input csv
wbHNP_df = pd.read_csv('../../Raw_Data/World_Bank-Extract_From_Health_Nutrition_and_Population_Statistics-1990_2022.csv')
wbHNP_df

,Series Name,Series Code,Country,Country Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Afghanistan,AFG,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Albania,ALB,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Algeria,DZA,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,American Samoa,ASM,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Andorra,AND,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Total alcohol consumption per capita (liters o...,SH.ALC.PCAP.LI,Virgin Islands (U.S.),VIR,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
864,Total alcohol consumption per capita (liters o...,SH.ALC.PCAP.LI,West Bank and Gaza,PSE,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
865,Total alcohol consumption per capita (liters o...,SH.ALC.PCAP.LI,"Yemen, Rep.",YEM,..,..,..,..,..,..,...,..,..,0.055,..,..,0.051,..,..,..,..
866,Total alcohol consumption per capita (liters o...,SH.ALC.PCAP.LI,Zambia,ZMB,..,..,..,..,..,..,...,..,..,6.89,..,..,6.54,..,..,..,..


In [3]:
# Drop the extra columns
wbHNP_df = wbHNP_df.drop(['Country Code','Series Code'], 1)
wbHNP_df

C:\Temp\ipykernel_3544\2268001833.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  wbHNP_df = wbHNP_df.drop(['Country Code','Series Code'], 1)


,Series Name,Country,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Capital health expenditure (% of GDP),Afghanistan,..,..,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),American Samoa,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Total alcohol consumption per capita (liters o...,Virgin Islands (U.S.),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
864,Total alcohol consumption per capita (liters o...,West Bank and Gaza,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
865,Total alcohol consumption per capita (liters o...,"Yemen, Rep.",..,..,..,..,..,..,..,..,...,..,..,0.055,..,..,0.051,..,..,..,..
866,Total alcohol consumption per capita (liters o...,Zambia,..,..,..,..,..,..,..,..,...,..,..,6.89,..,..,6.54,..,..,..,..


In [4]:
pattern = r'\[.*\]'

# Remove the bracketed year info from the column names
wbHNP_df.columns=[re.sub(pattern,'',col)for col in wbHNP_df.columns]
wbHNP_df.columns=wbHNP_df.columns.str.strip()
wbHNP_df

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Capital health expenditure (% of GDP),Afghanistan,..,..,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),American Samoa,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Total alcohol consumption per capita (liters o...,Virgin Islands (U.S.),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
864,Total alcohol consumption per capita (liters o...,West Bank and Gaza,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
865,Total alcohol consumption per capita (liters o...,"Yemen, Rep.",..,..,..,..,..,..,..,..,...,..,..,0.055,..,..,0.051,..,..,..,..
866,Total alcohol consumption per capita (liters o...,Zambia,..,..,..,..,..,..,..,..,...,..,..,6.89,..,..,6.54,..,..,..,..


In [5]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [6]:
# Apply the correction dictionary to fix the known errors
wbHNP_df['Country'] = wbHNP_df['Country'].replace(correction_dict)

In [7]:
wbHNP_df

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Capital health expenditure (% of GDP),Afghanistan,..,..,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),American Samoa,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Total alcohol consumption per capita (liters o...,Virgin Islands (U.S.),..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
864,Total alcohol consumption per capita (liters o...,West Bank and Gaza,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
865,Total alcohol consumption per capita (liters o...,Yemen,..,..,..,..,..,..,..,..,...,..,..,0.055,..,..,0.051,..,..,..,..
866,Total alcohol consumption per capita (liters o...,Zambia,..,..,..,..,..,..,..,..,...,..,..,6.89,..,..,6.54,..,..,..,..


In [8]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], 1)

C:\Temp\ipykernel_3544\1673923060.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  countries_df = countries_df.drop(['profile_url'], 1)


In [9]:
master_countries = countries_df['country'].tolist()
master_countries

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong (China)',
 

In [10]:
# Create a list of wbdi countries
ps_countries = wbHNP_df['Country'].tolist()
ps_countries

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Democratic Republic of the Congo',
 'Congo',
 'Costa Rica',
 "Côte d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 '

In [11]:
# clean the FS data based on the SS country list
no_match = []

for country in ps_countries:
    if country in master_countries:
        continue
    else:
        no_match.append(country)

In [12]:
# Likely need to do manual country name clean-up for accented characters and abbreviations.
no_match

['Afghanistan',
 'American Samoa',
 'Bosnia and Herzegovina',
 'Cayman Islands',
 'Channel Islands',
 'Comoros',
 'Curacao',
 'Eritrea',
 'Faroe Islands',
 'French Polynesia',
 'Gibraltar',
 'Greenland',
 'Guam',
 'Iraq',
 "Korea, Dem. People's Rep.",
 'Kosovo',
 'Macao SAR, China',
 'Mongolia',
 'Montenegro',
 'Nauru',
 'New Caledonia',
 'Northern Mariana Islands',
 'Puerto Rico',
 'Sint Maarten (Dutch part)',
 'Somalia',
 'South Sudan',
 'St. Martin (French part)',
 'Timor-Leste',
 'Tonga',
 'Turks and Caicos Islands',
 'Tuvalu',
 'United Arab Emirates',
 'Virgin Islands (U.S.)',
 'West Bank and Gaza',
 'Afghanistan',
 'American Samoa',
 'Bosnia and Herzegovina',
 'Cayman Islands',
 'Channel Islands',
 'Comoros',
 'Curacao',
 'Eritrea',
 'Faroe Islands',
 'French Polynesia',
 'Gibraltar',
 'Greenland',
 'Guam',
 'Iraq',
 "Korea, Dem. People's Rep.",
 'Kosovo',
 'Macao SAR, China',
 'Mongolia',
 'Montenegro',
 'Nauru',
 'New Caledonia',
 'Northern Mariana Islands',
 'Puerto Rico',
 'S

In [13]:
wbHNP_clean_df = wbHNP_df.drop(wbHNP_df[wbHNP_df['Country'].isin(no_match)].index.tolist())
wbHNP_clean_df

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
5,Capital health expenditure (% of GDP),Angola,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
6,Capital health expenditure (% of GDP),Antigua and Barbuda,..,..,..,..,..,..,..,..,...,..,..,0.04921389,0.14435445,0.00550222,0.02265624,0.02125437,0.00856461,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,Total alcohol consumption per capita (liters o...,Venezuela,..,..,..,..,..,..,..,..,...,..,..,6.82,..,..,4.14,..,..,..,..
862,Total alcohol consumption per capita (liters o...,Vietnam,..,..,..,..,..,..,..,..,...,..,..,7.86,..,..,8.66,..,..,..,..
865,Total alcohol consumption per capita (liters o...,Yemen,..,..,..,..,..,..,..,..,...,..,..,0.055,..,..,0.051,..,..,..,..
866,Total alcohol consumption per capita (liters o...,Zambia,..,..,..,..,..,..,..,..,...,..,..,6.89,..,..,6.54,..,..,..,..


In [14]:
# Open the list of series name update list
series_updates_df = pd.read_csv('../../Clean_Data/master_country_list/series_name_updates.csv')

# Convert the series_updates dataframe to a dictionary.
series_updates_dict = dict(zip(series_updates_df['current'], series_updates_df['updated']))

In [15]:
# Apply the series_updates dictionary to reduce verbosity
wbHNP_clean_df['Series Name'] = wbHNP_clean_df['Series Name'].replace(series_updates_dict)

In [16]:
wbHNP_clean_df

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Capital HE (% GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital HE (% GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
4,Capital HE (% GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
5,Capital HE (% GDP),Angola,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
6,Capital HE (% GDP),Antigua and Barbuda,..,..,..,..,..,..,..,..,...,..,..,0.04921389,0.14435445,0.00550222,0.02265624,0.02125437,0.00856461,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,Alcohol Consumption per capita (liters),Venezuela,..,..,..,..,..,..,..,..,...,..,..,6.82,..,..,4.14,..,..,..,..
862,Alcohol Consumption per capita (liters),Vietnam,..,..,..,..,..,..,..,..,...,..,..,7.86,..,..,8.66,..,..,..,..
865,Alcohol Consumption per capita (liters),Yemen,..,..,..,..,..,..,..,..,...,..,..,0.055,..,..,0.051,..,..,..,..
866,Alcohol Consumption per capita (liters),Zambia,..,..,..,..,..,..,..,..,...,..,..,6.89,..,..,6.54,..,..,..,..


In [17]:
wbHNP_clean_df = wbHNP_clean_df.set_index('Country').T
wbHNP_clean_df

Country,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
Series Name,Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),Capital HE (% GDP),...,Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters),Alcohol Consumption per capita (liters)
1990,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1991,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1992,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1993,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1994,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1995,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1996,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1997,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1998,..,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [18]:
wbHNP_clean_df.to_csv('../../Clean_Data/Clean_CSV_Files/world_bank_Health_Nutrition_and_Population_Statistics.csv', index=False)

In [19]:
#fsle_clean_transform_df = fsle_clean_df.set_index('Entity').T
#fsle_clean_transform_df

In [20]:
#fsle_clean_transform_df.to_csv('food_supply_life_expectancy_transform.csv')